In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
from torch.autograd import Function
def get_parameter_number(net):
    total_num = sum(p.numel() for p in net.parameters())
    trainable_num = sum(p.numel() for p in net.parameters() if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}

dtype = torch.float

class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, *size ,dtype = dtype)
        self.label = torch.rand(1, dtype = dtype)

    def __getitem__(self, index):
        return self.data[index], self.label

    def __len__(self):
        return self.len

        
def Random_DataLoader():
    input_size = [3]

    train_img_loader = DataLoader(dataset=RandomDataset(input_size, length = 100),
                         batch_size=2, shuffle=True)
    val_img_loader =  DataLoader(dataset=RandomDataset(input_size, length = 10),
                         batch_size=2, shuffle=False)

    return train_img_loader, val_img_loader


trainloader, valloader =  Random_DataLoader()

In [20]:
class LinearFunction(torch.autograd.Function):
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
            
        return output

    @staticmethod
    def backward(ctx, grad_output):
#         print(grad_output)
        input, weight, bias = ctx.saved_tensors
        
        grad_input = grad_weight = grad_bias = None

        
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)

            
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)

                 
        if bias is not None and ctx.needs_input_grad[2]:
            
            grad_bias = grad_output.sum(0).squeeze(0)
            grad_bias = grad_bias.view_as(bias)
#         print("*" * 40)
#         print(bias.size())
#         print(grad_weight.size())
#         print(grad_bias.size())
#         print("*" * 40)
        return grad_input, grad_weight, grad_bias



In [21]:
class Linear(nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        super(Linear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features
        self.weight = nn.Parameter(torch.Tensor(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            self.register_parameter('bias', None)
        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)
        
    def forward(self, input):
        # See the autograd section for explanation of what happens here.
        return LinearFunction.apply(input, self.weight, self.bias)

    def extra_repr(self):
        # (Optional)Set the extra information about this module. You can test
        # it by printing an object of this class.
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None)

In [22]:
demo_net = Linear(3,1)

criterion = nn.MSELoss()
optimizer = optim.SGD(demo_net.parameters(), lr=0.001, momentum=0.9)


for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        
        inputs, labels = data
        optimizer.zero_grad()

        outputs = demo_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,    20] loss: 0.006
[1,    40] loss: 0.003
Finished Training
